In [28]:
import plotly.plotly as py
import plotly.graph_objs as go
import networkx as nx
import pandas as pd
import numpy as np
import xlwings
import pyperclip
from collections import Counter
import pprint

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
df = pd.read_csv('Dump_DIPs_31_Jul_2018 (2).csv')
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
df = df.dropna(how='all')

In [29]:
#assigns values to each row in the column and removes blank rows
data = df.copy()
data = data.fillna({'case_name':'none'})
data['Case'] = 'xxxx'
value = 'Fees:'
case = 'xxxx'
for row in data.itertuples():
    if row[1] not in ['none', 'Fees:', 'Agents:','Lender Groups:', 'Roll ups:', 'Lenders:']:
        case = row[1]
        data.at[row[0], 'Case'] = case
        continue
    if row[1] == 'Fees:':
        value = 'Fees:'
        continue
    if row[1] == 'Agents:':
        value = 'Agents:'
        continue
    if row[1] == 'Lender Groups:':
        value = 'Lender Groups:'
        continue
    if row[1] == 'Roll ups:':
        value = 'Roll ups:'
        continue
    if row[1] == 'Lenders:':
        value = 'Lenders:'
        continue
    data.at[row[0], 'Case'] = case
    data.at[row[0], 'case_name'] = value
data = data[~data.Case.isin(['xxxx'])]
data = data.reset_index(drop=True)
#data = data.set_index('Case')

In [7]:
original_admin_list = list(data[data.petition_date.isin(['Admin Agent', 'Indenture Trustee', 'Lender'])]['Case'])
len(original_admin_list)

220

In [10]:
case_dict

{'In re Fairway Group Holdings Corp.': {'Case_Count': 2,
  'In re Fairway Group Holdings Corp.: 1': {'Data': [('Publishing_Name',
     'DIP Term Loan'),
    ('Currency', 'USD'),
    ('Loan_Amount', 55000000.0),
    ('Maturity', 3.0),
    ('Facility_Type', 'Term Loan'),
    ('Fixed_Float', 'Floating'),
    ('Floating_Type', 'LIBOR'),
    ('Investigation_Budget', 250000.0),
    ('Carve_Out', 1000000.0)],
   'Fees': [('Type', 'Arrangement/Origination Fee'),
    ('Sub_Type', nan),
    ('Amount', '0.5'),
    ('Amount_Type', '%'),
    ('Duration_Type', 'One-time')],
   'Agents': {'Admin Agent': ['Credit Suisse'],
    'Collateral Agent': ['Credit Suisse']}},
  'In re Fairway Group Holdings Corp.: 2': {'Data': [('Publishing_Name',
     'DIP Revolver'),
    ('Currency', 'USD'),
    ('Loan_Amount', 30611941.0),
    ('Maturity', 3.0),
    ('Facility_Type', 'Revolver'),
    ('Fixed_Float', 'Floating'),
    ('Floating_Type', 'LIBOR'),
    ('Investigation_Budget', 25000.0),
    ('Carve_Out', 1000000

In [30]:
starting_list = list(data['case_name'])
clean_list = [x for x in starting_list if str(x) != 'nan']
unique_list = ['Agents:', 'Fees:', 'Lender Groups:', 'Lenders:', 'Roll ups:']
all_cases = [x for x in clean_list if str(x) not in ['Agents:', 'Fees:', 'Lender Groups:', 'Lenders:', 'Roll ups:']]
total_cases = len(all_cases)
#counts the total number of cases present in the dataframe
#observation_count = Counter(clean_list) #dictionary of count of unique deals, Fees, Agents ect.

#combs through the created list and returns the case names that have missing data i.e No Agents, No Lenders ect.
master_list = ['Agents:', 'Fees:', 'Lender Groups:', 'Lenders:', 'Roll ups:']
case_dict = {}
case_index = []
agent_no_lenders = []
no_data = []
lenders_no_agent = []
one_lender_no_agent = []
no_agent = []

for i in range(0, len(clean_list)):
    if clean_list[i] not in master_list:
        fee_num = 0
        agent_num = 0
        name = clean_list[i]
        case_index.append(i)
        dup_case = name in case_dict
        if not dup_case:
            case_dict[name] = {}
            case_dict[name]['Case_Count'] = 1
            dip_num = name + ': ' + str(1)
        if dup_case:
            case_dict[name]['Case_Count'] = case_dict[name]['Case_Count'] + 1
            dip_num = name + ': ' + str(case_dict[name]['Case_Count'])
        case_dict[name][dip_num] = {}
        case_dict[name][dip_num]['Data'] = {}
        case_dict[name][dip_num]['Data']['Publishing_Name'] = data.loc[i]['publishing_name']
        case_dict[name][dip_num]['Data']['Currency'] = data.loc[i]['currency']
        case_dict[name][dip_num]['Data']['Loan_Amount'] = data.loc[i]['loan_amount']
        case_dict[name][dip_num]['Data']['Maturity'] = data.loc[i]['maturity']
        case_dict[name][dip_num]['Data']['Facility_Type'] = data.loc[i]['facility_type']
        case_dict[name][dip_num]['Data']['Fixed_Float'] = data.loc[i]['fixed/floating']
        case_dict[name][dip_num]['Data']['Floating_Type'] = data.loc[i]['floating_type']
        case_dict[name][dip_num]['Data']['Investigation_Budget'] = data.loc[i]['investigation_budget']
        case_dict[name][dip_num]['Data']['Carve_Out'] = data.loc[i]['carve_out']
    if clean_list[i] == 'Fees:':
        if fee_num == 0:
            case_dict[name][dip_num]['Fees'] = {}
        fee_num = fee_num + 1
        fee_name = 'Fee_'+str(fee_num)
        case_dict[name][dip_num]['Fees'][fee_name] = {}
        case_dict[name][dip_num]['Fees'][fee_name]['Type'] = data.loc[i]['case_status']
        case_dict[name][dip_num]['Fees'][fee_name]['Sub_Type'] = data.loc[i]['petition_date']
        case_dict[name][dip_num]['Fees'][fee_name]['Amount'] = data.loc[i]['confirmation_date']
        case_dict[name][dip_num]['Fees'][fee_name]['Amount_Type'] = data.loc[i]['dismissal_date']
        case_dict[name][dip_num]['Fees'][fee_name]['Duration_Type'] = data.loc[i]['closing_date']
    if clean_list[i] == 'Agents:':
        agent_type = str(data.loc[i]['petition_date'])
        if str(data.loc[i]['petition_date']) in ['Lender', 'Indenture Trustee']:
            agent_type = 'Admin Agent'
        if 'Agents' not in case_dict[name][dip_num]:
            case_dict[name][dip_num]['Agents'] = {}
            case_dict[name][dip_num]['Agents'][agent_type] = []
        if 'Agents' in case_dict[name][dip_num]:
            if agent_type not in case_dict[name][dip_num]['Agents']:
                case_dict[name][dip_num]['Agents'][agent_type] = []
        case_dict[name][dip_num]['Agents'][agent_type].append(str(data.loc[i]['case_status']))
    if clean_list[i] == 'Lenders:':
        lender = str(data.loc[i]['case_status'])
        if 'Lenders' not in case_dict[name][dip_num]:
            case_dict[name][dip_num]['Lenders'] = []
            case_dict[name][dip_num]['Lenders'].append(lender)
    

In [19]:
case_dict

{'In re Fairway Group Holdings Corp.': {'Case_Count': 2,
  'In re Fairway Group Holdings Corp.: 1': {'Data': {'Publishing_Name': 'DIP Term Loan',
    'Currency': 'USD',
    'Loan_Amount': 55000000.0,
    'Maturity': 3.0,
    'Facility_Type': 'Term Loan',
    'Fixed_Float': 'Floating',
    'Floating_Type': 'LIBOR',
    'Investigation_Budget': 250000.0,
    'Carve_Out': 1000000.0},
   'Fees': {'Fee_1': {'Type': ' Unused Commitment Fee',
     'Sub_Type': nan,
     'Amount': '0.375',
     'Amount_Type': '%',
     'Duration_Type': 'Monthly'},
    'Fee_2': {'Type': 'Other Fee',
     'Sub_Type': 'Duration Fee',
     'Amount': '0.25',
     'Amount_Type': '%',
     'Duration_Type': 'One-time'},
    'Fee_3': {'Type': 'Commitment Fee',
     'Sub_Type': nan,
     'Amount': '2',
     'Amount_Type': '%',
     'Duration_Type': 'One-time'},
    'Fee_4': {'Type': 'Arrangement/Origination Fee',
     'Sub_Type': nan,
     'Amount': '0.5',
     'Amount_Type': '%',
     'Duration_Type': 'One-time'}},
   'A

In [39]:

dict_count = 0
admin_count = 0
no_admin_agent = []
admin_name = []
all_admin = []
multiple_admin = []
single_lender = []
no_agent = []
no_agent_and_lender = []
for i in case_dict.keys():
    dict_count += case_dict[i]['Case_Count']
    for j in range(1, case_dict[i]['Case_Count']+1):
        dip_number = str(i) + ': ' + str(j)
        if 'Agents' not in case_dict[i][dip_number]:
            no_agent.append(i)
            if 'Lenders' in case_dict[i][dip_number]:
                if len(case_dict[i][dip_number]['Lenders']) == 1:
                    single_lender.append(i)
            if 'Lenders' not in case_dict[i][dip_number]:
                no_agent_and_lender.append(i)
        if 'Agents' in case_dict[i][dip_number]:
            if 'Admin Agent' in case_dict[i][dip_number]['Agents']:
                if len(case_dict[i][dip_number]['Agents']['Admin Agent']) > 1:
                    multiple_admin.append(i)
                admin_name.append(case_dict[i][dip_number]['Agents']['Admin Agent'][0])
                all_admin.append(i)
                admin_count += 1
            else:
                no_admin_agent.append(dip_number)
                continue

print('Case Count: ', total_cases)
print('Dictionary Case Count: ', dict_count)
print('No Admin Agent Count: ', len(no_admin_agent))
#print('Admin Agent Total: ', admin_agent_total)
print('Dictionary Admin Agent Count: ', admin_count)
print('Multiple Admin Agents in Dictionary Count: ', len(multiple_admin))
print('Single Lender, No Agent Count: ', len(single_lender))
print('No Agent Listed Count: ', len(no_agent))
print('No Agent and Lender Count: ', len(no_agent_and_lender))

Case Count:  348
Dictionary Case Count:  348
No Admin Agent Count:  2
Dictionary Admin Agent Count:  215
Multiple Admin Agents in Dictionary Count:  5
Single Lender, No Agent Count:  128
No Agent Listed Count:  131
No Agent and Lender Count:  3


In [40]:
no_admin_agen

['In re Katy Industries, Inc.',
 'In re Mountain Divide, LLC (Mountainview Energy)',
 'In re Pacific Exploration &amp; Production Corporation']

In [ ]:
'''
Missing some data because there are no Admin agents listed and no lenders listed for some of these cases.
'''

In [46]:
case_index = []
dip_number_index = []
data_pull = []
#key_list = list(case_dict.keys()
for i in list(case_dict.keys()):
    for j in range(1, case_dict[i]['Case_Count']+1):
        dip_number = str(i) + ': ' + str(j)
        if 'Agents' in case_dict[i][dip_number]:
            if 'Admin Agent' in case_dict[i][dip_number]['Agents']:
                case_index.append(i)
                dip_number_index.append(dip_number)
                data_pull.append([case_dict[i][dip_number]['Data']['Loan_Amount'], case_dict[i][dip_number]['Agents']['Admin Agent'][0]])
        if 'Agents' not in case_dict[i][dip_number]:
            if 'Lenders' not in case_dict[i][dip_number]:
                continue
            if len(case_dict[i][dip_number]['Lenders']) == 1:
                case_index.append(i)
                dip_number_index.append(dip_number)
                data_pull.append([case_dict[i][dip_number]['Data']['Loan_Amount'], case_dict[i][dip_number]['Lenders'][0]])
print('Number of Rows: ', len(case_index))

Number of Rows:  343


In [45]:
pd.DataFrame(data=data_pull, index = [case_index, dip_number_index], columns=['loan_amount', 'admin_agent']).to_csv('Agents_Loan_Amount.csv',encoding='utf-8')